In [1]:
import numpy as np
import pandas as pd


In [2]:
file_loc=r"C:\Users\Naveen Reddy\Desktop\Practice\Encoder Decoder\fra-eng\fra.txt"
data=pd.read_csv(file_loc,delimiter="\t",header=None)
data.drop(2,inplace=True,axis=1)
data.columns=["English","French"]

In [3]:
english_data,french_data=list(data["English"]),list(data["French"])
english_data,french_data=english_data[:95103],french_data[:95103]

In [4]:
eng_char=set()
for i in english_data:
    eng_char=eng_char.union(set(list(i)))
eng_char=list(eng_char)
eng_char=sorted(eng_char)
eng_char=dict([(j,i) for i,j in enumerate(eng_char)])


In [5]:
french_char=set()
for i in french_data:
    i="/s"+i+"/e"
    french_char=french_char.union(set(list(i)))
lis=[]
french_char=list(french_char)
french_char=sorted(french_char)
french_char=dict([(j,i) for i,j in enumerate(french_char)])
    

In [6]:
max_eng_len=int(max([len(list(i)) for i in english_data]))
print(max_eng_len)
max_fren_len=int(max([len(list(i)) for i in french_data]))
print(max_fren_len)

28
76


In [7]:
num_eng_sentence=len(english_data)
num_fren_sentence=len(french_data)
print(num_eng_sentence)
print(num_fren_sentence)

95103
95103


In [8]:
encoder_input=np.zeros((num_eng_sentence,max_eng_len,len(eng_char)),dtype=np.float32)
decoder_input=np.zeros((num_fren_sentence,max_fren_len,len(french_char)),dtype=np.float32)
decoder_output=np.zeros((num_fren_sentence,max_fren_len,len(french_char)),dtype=np.float32)


In [9]:
for i in range(len(english_data)):
  for j in range(len(list(english_data[i]))):
    if list(english_data[i])[j] in eng_char:
      encoder_input[i,j,eng_char[list(english_data[i])[j]]]=1

In [10]:
for i in range(len(french_data)):
  for j in range(len(list(french_data[i]))):
    if list(french_data[i])[j-1] in french_char:
      decoder_output[i,j-1,french_char[list(french_data[i])[j-1]]]=1
    if list(french_data[i])[j] in french_char:
      decoder_input[i,j,french_char[list(french_data[i])[j]]]=1


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
encode_inputs=Input(shape=(None,len(eng_char)))
encoder=LSTM(256,return_state=True)
_,encode_state1,encode_state2=encoder(encode_inputs)
enc_states=[encode_state1,encode_state2]
decode_inputs=Input(shape=(None,len(french_char)))
decoder=LSTM(256,return_sequences=True,return_state=True)
dec_outputs,_,_=decoder(decode_inputs,initial_state=enc_states)
decoder_dense=Dense(len(french_char),activation="softmax")
decode_outs=decoder_dense(dec_outputs)

In [12]:
model=Model([encode_inputs,decode_inputs],decode_outs)
model.compile(loss='categorical_crossentropy')
model.fit([encoder_input,decoder_input],decoder_output,epochs=5)

Epoch 1/5
2972/2972 [==============================] - 485s 163ms/step - loss: 0.4676
Epoch 2/5
2972/2972 [==============================] - 585s 197ms/step - loss: 0.3153
Epoch 3/5
2972/2972 [==============================] - 596s 201ms/step - loss: 0.2987
Epoch 4/5
2972/2972 [==============================] - 1425s 480ms/step - loss: 0.2873
Epoch 5/5
2972/2972 [==============================] - 743s 250ms/step - loss: 0.2748


In [28]:
pred=model.predict([encoder_input,decoder_input])
print(pred)

[[[5.37998090e-03 1.91536651e-03 2.71962746e-03 ... 2.89308513e-03
   3.42035014e-03 2.53187888e-03]
  [7.11459259e-04 1.15664629e-03 7.65150122e-04 ... 1.18722732e-04
   4.48109582e-04 7.12415203e-04]
  [5.95105767e-01 4.20017354e-03 1.01532019e-03 ... 5.39364177e-04
   1.28379837e-03 4.10989486e-03]
  ...
  [7.33280927e-02 1.34785045e-02 7.20992510e-04 ... 5.80891734e-04
   1.77795056e-03 1.09419068e-02]
  [7.33280927e-02 1.34785045e-02 7.20992510e-04 ... 5.80891734e-04
   1.77795056e-03 1.09419068e-02]
  [7.33280927e-02 1.34785045e-02 7.20992510e-04 ... 5.80891734e-04
   1.77795056e-03 1.09419068e-02]]

 [[3.16265528e-03 1.26571918e-03 4.29405714e-04 ... 6.39519771e-04
   7.32631481e-04 2.34598643e-03]
  [7.18802330e-04 1.78449042e-03 5.57145104e-04 ... 1.25031162e-04
   3.60203238e-04 5.84168185e-04]
  [2.49277502e-02 1.16549125e-02 1.97309745e-03 ... 6.64302032e-04
   1.37207401e-03 1.31874140e-02]
  ...
  [7.33280927e-02 1.34785045e-02 7.20992510e-04 ... 5.80891734e-04
   1.77795

In [29]:
pred.shape  ## 57- padded words, 100- number of charecters(loop)

(95103, 76, 105)

In [30]:
pred1=np.moveaxis(pred,2,1)
pred1.shape
reverse={n:a for a,n in french_char.items()}

In [31]:
emp=''
lis2=[]
for i in pred[:10]:
  for j in i:
    emp=emp+reverse[np.argmax(j)]
  lis2.append(emp)

In [32]:
lis2

['Va                                                                          ',
 'Va                                                                          Marc e                                                                      ',
 'Va                                                                          Marc e                                                                      Bouge                                                                       ',
 'Va                                                                          Marc e                                                                      Bouge                                                                       Saiut                                                                       ',
 'Va                                                                          Marc e                                                                      Bouge                                                             

In [19]:
model.save("model1")

INFO:tensorflow:Assets written to: model1\assets


INFO:tensorflow:Assets written to: model1\assets
